### Q1. Running Elastic

In [1]:
! curl localhost:9200

{
  "name" : "025a6d95ab4d",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "gtiCqXoIQ6iwSmDSx2UuGw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [92]:
# docker run -it --rm --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [2]:
import requests
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()
documents = []
for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

index

### Q3. Searching

In [3]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import requests
 
# Create an Elasticsearch client instance
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}], request_timeout=30)
 
# URL of the documents JSON file
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()
 
# Prepare documents for bulk indexing
documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append({
            "_index": "documents_index",
            "_source": doc
        })
 


In [4]:
# Define the index settings and mappings
index_name = "documents_index"
index_body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}
# Create the index with the defined settings and mappings
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_body)

In [6]:
# Index documents using bulk API
bulk(es, documents)
 
# Refresh the index to make sure all documents are searchable
# es.indices.refresh(index="documents_index")
print(es.indices.exists(index="documents_index"))
# Define the search query
# query = {
#     "query": {
#         "multi_match": {
#             "query": "How do I execute a command in a running docker container?",
#             "fields": ["question^4", "text"],
#             "type": "best_fields"
#         }
#     }
# }
query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I execute a command in a running docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
# Execute the search query
response = es.search(index="documents_index", body=query)
print(response)
# Get the score for the top-ranking result
top_score = response['hits']['hits'][0]['_score']
print(f"Top score: {top_score}")

True
{'took': 40, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'documents_index', '_id': 'vFjASZAByWajB53U-iDv', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}]}}
Top score: 84.050095


In [31]:
# result = es.search(index=index_name, body={"query": {"match_all": {}}})
# print(len(result['hits']['hits']))
# es.indices.delete(index=index_name)

10


ObjectApiResponse({'acknowledged': True})

### Q4. Filtering

In [7]:
# Index documents using bulk API
# bulk(es, documents)

# Refresh the index to make sure all documents are searchable
# es.indices.refresh(index="documents_index")
# Define the search query
q4_search_query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I execute a command in a running docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"                
                }            
            },            
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }        
        }    
    },    
    "size": 3
}

# Perform the search
response = es.search(index="documents_index", body=q4_search_query)

for hit in response['hits']['hits']:
    print("\n", hit['_score'], hit['_source'])
# print(response['hits']['hits']['_score'])
# Extract the 3rd question from the search results
third_question = response['hits']['hits'][2]['_source']['question']
print(f"\n\nThe 3rd question returned by the search engine is: {third_question}")



 84.17781 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}

 84.17781 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}

 51.134113 {'text': "You ca

In [8]:
# result = es.delete_by_query(index=index_name, body={"query": {"match_all": {}}})
# print(result)

### Q5. Building a prompt

In [9]:
results = [{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'machine-learning-zoomcamp'},{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from a different folder into docker container’s working directory?', 'course': 'machine-learning-zoomcamp'}]
context = ""
context_template = context_template = """
Q: {question}
A: {text}
""".strip()
for doc in results:
    context = context + context_template.format(question=doc['question'], text=doc['text']).strip() + "\n\n"
#print(context)


question = "How do I execute a command in a running docker container?"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".format(question=question, context=context).strip()

print(prompt_template)
print(len(prompt_template))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

### Q6. Tokens

In [12]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
encoded_prompt = encoding.encode(prompt_template)

# Print the encoded tokens
print(encoded_prompt)
print(len(encoded_prompt))

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480, 734, 1385, 51